In [1]:
import findspark
findspark.init('/home/diegofvargas/spark-2.4.4-bin-hadoop2.7')
import pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [3]:
data = spark.read.csv('smsspamcollection/SMSSpamCollection',inferSchema=True,sep='\t')

In [4]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [5]:
data= data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [6]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [7]:
from pyspark.sql.functions import length

In [8]:
data = data.withColumn('length',length(data['text']))

In [9]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [16]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [17]:
# Created a feature since it seems like the average count for each class could help in the predictions

In [11]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,IDF,StringIndexer

In [12]:
#Create tokens out of sentences to split the strings
tokenizer = Tokenizer(inputCol='text',outputCol='token_text')
# Get rid of stop words that don't add value
stop_remove = StopWordsRemover(inputCol='token_text',outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token',outputCol='c_vec')
idf = IDF(inputCol='c_vec',outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='class',outputCol='label')

In [13]:
from pyspark.ml.feature import VectorAssembler

In [14]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

In [18]:
from pyspark.ml.classification import NaiveBayes

In [19]:
nb = NaiveBayes()

In [20]:
from pyspark.ml import Pipeline

In [22]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_numeric,tokenizer,stop_remove,count_vec,idf,clean_up])

In [23]:
clean_data = data_prep_pipe.fit(data).transform(data)

In [24]:
clean_data = clean_data.select('label','features')

In [25]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [26]:
training,test = clean_data.randomSplit([0.7,0.3])

In [34]:
# Naive Bayes 

In [27]:
spam_detector = nb.fit(training)

In [28]:
test_results = spam_detector.transform(test)

In [29]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,4,50,...|[-827.81229568250...|[1.0,4.1961514137...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-871.92543797720...|[1.0,5.1574923317...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-1154.1708385246...|[1.0,2.1037107743...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-540.32832369607...|[1.0,2.5180511044...|       0.0|
|  0.0|(13424,[0,1,11,32...|[-888.85356319055...|[1.0,4.8819682976...|       0.0|
|  0.0|(13424,[0,1,12,33...|[-447.22988054023...|[1.0,4.1050566921...|       0.0|
|  0.0|(13424,[0,1,14,78...|[-706.77273409743...|[1.0,4.2418323678...|       0.0|
|  0.0|(13424,[0,1,20,27...|[-972.99665924833...|[0.99999999997554...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-996.59174714556...|[1.0,4.4865728349...|       0.0|
|  0.0|(13424,[0

In [30]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [31]:
acc_eval = MulticlassClassificationEvaluator()

In [32]:
acc = acc_eval.evaluate(test_results)

In [33]:
print(acc)

0.9197404185567996


In [35]:
#Logistic Regression

In [36]:
from pyspark.ml.classification import LogisticRegression

In [37]:
lr = LogisticRegression()

In [38]:
spam_detector_lr = lr.fit(training)

In [39]:
test_results_lr = spam_detector_lr.transform(test)

In [40]:
acc_lr = acc_eval.evaluate(test_results_lr)

In [41]:
print(acc_lr)

0.9756198181436383


In [42]:
#Random Forest

In [43]:
from pyspark.ml.classification import RandomForestClassifier

In [49]:
rf = RandomForestClassifier()

In [50]:
spam_detector_rf = rf.fit(training)

In [51]:
test_results_rf = spam_detector_rf.transform(test)

In [52]:
acc_rf = acc_eval.evaluate(test_results_rf)

In [53]:
print(acc_rf)

0.7945500061985817
